# 06. Final Result Generation


In [ ]:
import time
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.optimize import differential_evolution
import torch
from scipy.stats import norm
from scipy.optimize import brentq

# =========================================================
# 1. ⚛️ The Engine: Threshold Bates Model (GPU Accelerated)
# =========================================================
def simulate_threshold_bates(S0, T, dt, num_paths, params, device='cuda'):
    # --- Heston Parameters ---
    kappa = float(params['kappa'])
    theta = float(params['theta'])
    xi    = float(params['xi'])
    rho   = float(params['rho'])
    
    # --- Regime Switching Parameters ---
    mu  = float(params['mu'])      # Jump Mean
    sig = float(params['sig'])     # Jump Vol
    thresh   = float(params['thresh'])   # ⚡ The Critical Point (임계값)
    lam_low  = float(params['lam_low'])  # Normal State Jump Intensity
    lam_high = float(params['lam_high']) # Panic State Jump Intensity
    
    N_steps = int(T / dt)
    
    # Initialize Particles
    S = torch.full((num_paths,), float(S0), device=device, dtype=torch.float32)
    v = torch.full((num_paths,), float(theta), device=device, dtype=torch.float32)
    
    # Random Number Generation
    Z1 = torch.randn((num_paths, N_steps), device=device)
    Z2 = torch.randn((num_paths, N_steps), device=device)
    
    corr_factor = np.sqrt(1 - rho**2 + 1e-6)
    dt_sqrt = np.sqrt(dt)
    
    # Time Evolution (Path Integral)
    for t in range(N_steps):
        # 1. Stochastic Volatility (Heston)
        W_S = Z1[:, t]
        W_v = rho * W_S + corr_factor * Z2[:, t]
        
        v_abs = torch.abs(v)
        sqrt_v = torch.sqrt(v_abs)
        dv = kappa * (theta - v) * dt + xi * sqrt_v * W_v * dt_sqrt
        v = v + dv
        
        # 2. Diffusion Process
        dS_diff = (0 - 0.5 * v_abs) * dt + sqrt_v * W_S * dt_sqrt
        
        # 3. ⚡ Regime Switching Logic (Phase Transition)
        # 변동성이 임계값(thresh)을 넘으면 -> 고위험 모드(lam_high) 발동
        current_lam = torch.where(v_abs > thresh, 
                                  torch.tensor(lam_high, device=device), 
                                  torch.tensor(lam_low, device=device))
        
        prob_jump = current_lam * dt
        rand_uni = torch.rand(num_paths, device=device)
        is_jump = rand_uni < prob_jump
        
        # 4. Jump Process
        if is_jump.any():
            n_jumps = is_jump.sum()
            jump_vals = mu + sig * torch.randn(n_jumps, device=device)
            jump_vec = torch.zeros(num_paths, device=device)
            jump_vec[is_jump] = jump_vals
            S = S * torch.exp(dS_diff + jump_vec)
        else:
            S = S * torch.exp(dS_diff)
            
    return S

# =========================================================
# 2. 🛡️ The Judge: Defensive Loss Function (Manual Weighting)
# =========================================================
def implied_vol_cpu(prices, S, K_array, T):
    ivs = []
    for i, p in enumerate(prices):
        def obj(sigma):
            d1=(np.log(S/K_array[i])+0.5*sigma**2*T)/(sigma*np.sqrt(T))
            d2=d1-sigma*np.sqrt(T)
            return S*norm.cdf(d1)-K_array[i]*norm.cdf(d2)-p
        try: ivs.append(brentq(obj, 0.001, 3.0))
        except: ivs.append(np.nan)
    return np.array(ivs)

def final_loss(params, strikes_gpu, market_ivs, S0, T, num_paths):
    p_dict = {
        'kappa':params[0], 'theta':params[1], 'xi':params[2], 'rho':params[3],
        'thresh':params[4], 'lam_low':params[5], 'lam_high':params[6], 'mu':params[7], 'sig':params[8]
    }
    
    # Constraints (물리학적 제약조건)
    if params[0]<0 or params[1]<0 or params[2]<0 or abs(params[3])>0.99: return 1e9
    if params[4]<0 or params[5]<0 or params[6]<0 or params[8]<0: return 1e9
    if params[6] < params[5]: return 1e9 # High Regime Risk must be >= Low Regime Risk

    S_T = simulate_threshold_bates(S0, T, 1/252, num_paths, p_dict, device='cuda')
    S_T = S_T * (S0 / S_T.mean()) # Martingale Correction
    
    payoffs = torch.maximum(S_T.unsqueeze(1) - strikes_gpu.unsqueeze(0), torch.tensor(0.0, device='cuda'))
    prices = torch.mean(payoffs, dim=0).cpu().numpy()
    strikes_cpu = strikes_gpu.cpu().numpy()
    model_ivs = implied_vol_cpu(prices, S0, strikes_cpu, T)
    
    mask = ~np.isnan(model_ivs)
    if np.sum(mask) == 0: return 1e9
    
    # --- 🛡️ Defensive Weighting (Winner Logic) ---
    # 하방 리스크(Put)에 2배 가중치, 상방(Call)에 1.2배 가중치
    w = np.ones_like(strikes_cpu)
    w[strikes_cpu < S0] = 2.0 
    w[strikes_cpu > S0*1.05] = 1.2
    
    err = np.sum(w[mask]*(model_ivs[mask]-market_ivs[mask])**2)
    return float(np.sqrt(err/np.sum(w[mask])))

# =========================================================
# 3. 🚀 Execution: Finding the Hidden Physics of QQQ
# =========================================================
print("\n" + "="*70)
print("🧬 [Thesis Final Result] Extracting Phase Transition Parameters")
print("="*70)

def run_final_extraction(ticker):
    # 1. Data Loading
    t_obj = yf.Ticker(ticker)
    try: curr_p = t_obj.history(period="1d")['Close'].iloc[-1]
    except: return
    
    # Target Expiration Selection (약 1달 뒤 만기)
    exps = t_obj.options; tgt_date = exps[3]
    for e in exps:
        if 30 <= (datetime.strptime(e, "%Y-%m-%d") - datetime.now()).days <= 60: tgt_date = e; break
    
    calls = t_obj.option_chain(tgt_date).calls
    calls = calls[(calls['impliedVolatility']>0.01)&(calls['impliedVolatility']<1.0)]
    
    # Forward Price Calculation
    puts = t_obj.option_chain(tgt_date).puts
    atm_idx = (calls['strike'] - curr_p).abs().idxmin()
    atm_K = calls.loc[atm_idx, 'strike']
    try:
        c_p = (calls[calls['strike']==atm_K]['bid'].values[0] + calls[calls['strike']==atm_K]['ask'].values[0])/2
        p_p = (puts[puts['strike']==atm_K]['bid'].values[0] + puts[puts['strike']==atm_K]['ask'].values[0])/2
        S0_forward = atm_K + (c_p - p_p)
    except:
        S0_forward = curr_p

    T_v = (datetime.strptime(tgt_date, "%Y-%m-%d")-datetime.now()).days/365.0
    
    # Sampling for Optimization
    sample_rate = 4
    m_strikes = calls['strike'].values[::sample_rate]
    m_ivs = calls['impliedVolatility'].values[::sample_rate]
    m_strikes_gpu = torch.tensor(m_strikes, device='cuda', dtype=torch.float32)
    
    print(f"Target: {tgt_date} (T={T_v:.4f}) | Forward Price: ${S0_forward:.2f}")
    print("Optimization started... (Finding Global Optima)")

    # 2. Optimization (Genetic Algorithm)
    # Bounds: [kappa, theta, xi, rho, thresh, lam_low, lam_high, mu, sig]
    bounds = [(0.1, 10), (0.01, 0.5), (0.1, 5.0), (-0.99, -0.3), # Heston
              (0.01, 0.3), (0.01, 5.0), (1.0, 20.0),             # Regime (Thresh, Low, High)
              (-0.5, 0.05), (0.01, 0.5)]                         # Jump Size
    
    result = differential_evolution(
        lambda p: final_loss(p, m_strikes_gpu, m_ivs, S0_forward, T_v, 2000),
        bounds, strategy='best1bin', maxiter=10, popsize=12, polish=True, workers=1, disp=True
    )
    
    # 3. Final Visualization & Report
    p = result.x
    rmse = result.fun
    
    # Generate Smooth Curve
    N_sim = 30000
    strikes_all = calls['strike'].values
    strikes_gpu_all = torch.tensor(strikes_all, device='cuda', dtype=torch.float32)
    
    p_dict = {'kappa':p[0], 'theta':p[1], 'xi':p[2], 'rho':p[3], 
              'thresh':p[4], 'lam_low':p[5], 'lam_high':p[6], 'mu':p[7], 'sig':p[8]}
    
    S = simulate_threshold_bates(S0_forward, T_v, 1/252, N_sim, p_dict, device='cuda')
    S = S * (S0_forward / S.mean())
    payoffs = torch.maximum(S.unsqueeze(1) - strikes_gpu_all.unsqueeze(0), torch.tensor(0.0, device='cuda'))
    prices = torch.mean(payoffs, dim=0).cpu().numpy()
    model_ivs = implied_vol_cpu(prices, S0_forward, strikes_all, T_v)
    
    # --- 📊 Graph ---
    plt.figure(figsize=(12, 7))
    plt.scatter(calls['strike'], calls['impliedVolatility'], c='gray', alpha=0.5, label='Real Market Data')
    plt.plot(strikes_all, model_ivs, 'g-', linewidth=2.5, label=f'Threshold Bates (Manual-Weighted)\nRMSE={rmse:.4f}')
    plt.axvline(S0_forward, color='k', linestyle=':', label='Forward Price')
    plt.axvspan(calls['strike'].min(), S0_forward, alpha=0.1, color='red', label='Put Option Zone (Weighted 2.0)')
    
    plt.title(f"[{ticker}] Phase Transition Model Calibration", fontsize=15)
    plt.xlabel("Strike Price"); plt.ylabel("Implied Volatility")
    plt.legend(loc='upper center'); plt.grid(True, alpha=0.3)
    plt.show()

    # --- 📝 Table for Thesis ---
    print("\n" + "="*50)
    print("🏆 [Table 1] Final Estimated Parameters")
    print("="*50)
    print(f"{'Parameter':<20} | {'Value':<10} | {'Meaning'}")
    print("-" * 50)
    print(f"{'Threshold (K)':<20} | {np.sqrt(p[4]):.4f}     | Critical Volatility Level (approx {np.sqrt(p[4])*100:.1f}%)")
    print(f"{'Lambda (Low)':<20} | {p[5]:.4f}     | Jump Intensity in Normal State")
    print(f"{'Lambda (High)':<20} | {p[6]:.4f}     | Jump Intensity in Panic State")
    print(f"{'Regime Ratio':<20} | {p[6]/p[5]:.2f}x      | Risk Multiplier (Panic/Normal)")
    print("-" * 50)
    print(f"{'Kappa (Mean Rev)':<20} | {p[0]:.4f}")
    print(f"{'Theta (Long Vol)':<20} | {p[1]:.4f}")
    print(f"{'Xi (Vol of Vol)':<20} | {p[2]:.4f}")
    print(f"{'Rho (Correlation)':<20} | {p[3]:.4f}")
    print(f"{'Jump Mean (Mu)':<20} | {p[7]:.4f}")
    print(f"{'Jump Sig (Sigma)':<20} | {p[8]:.4f}")
    print("-" * 50)
    print(f"Final RMSE: {rmse:.5f}")
    print("="*50)
    print("👉 Thesis Interpretation:")
    print(f"1. Market implies a 'Critical Volatility' of {np.sqrt(p[4])*100:.1f}%.")
    print(f"2. When volatility crosses this line, crash probability increases by {p[6]/p[5]:.1f} times.")
    print("3. This proves the existence of 'Phase Transition' in QQQ options.")

run_final_extraction("QQQ")